# Практическая часть 2
Кластеризация методом k-средних и оценка качества (Silhouette)

## Блок 0. Подготовка окружения

Импортировать библиотеки для работы с данными, кластеризации и оценки качества.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.decomposition import PCA

%matplotlib inline

pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

## Блок 1. Подготовка данных для кластеризации

### 1.1. Загрузка набора данных Iris и выбор признаков

Загрузить набор данных Iris.
Для кластеризации использовать только числовые признаки (без целевой переменной).

In [ ]:
iris = datasets.load_iris(as_frame=True)
df_iris = iris.frame.copy()
df_iris["species"] = df_iris["target"].map(dict(enumerate(iris.target_names)))

X = df_iris[iris.feature_names]
X.head()

### 1.2. Базовая информация о данных

Проверить форму матрицы признаков и отсутствие пропусков.

In [ ]:
print("Форма X:", X.shape)
print("Пропуски по столбцам:")
print(X.isna().sum())

## Блок 2. Метод k-средних при фиксированном числе кластеров

### 2.1. Обучение k-means при k = 3

Применить метод k-средних с числом кластеров k = 3.
Сохранить номера кластеров для каждого объекта в отдельную колонку исходного DataFrame.

In [ ]:
k = 3
kmeans_3 = KMeans(n_clusters=k, random_state=42, n_init=10)
cluster_labels_3 = kmeans_3.fit_predict(X)

df_iris["cluster_k3"] = cluster_labels_3
df_iris.head()

### 2.2. Сводная таблица «кластеры ↔ реальные классы»

Построить перекрёстную таблицу частот:
строки — реальные виды ирисов (species),
столбцы — кластеры k-means.

In [ ]:
ct_k3 = pd.crosstab(df_iris["species"], df_iris["cluster_k3"])
ct_k3

### 2.3. Визуализация кластеров на исходных признаках

Построить диаграмму рассеяния по двум выбранным признакам (например, длина и ширина лепестка).
Раскрасить точки по кластерам k-means.

In [ ]:
plt.figure(figsize=(6, 5))
plt.scatter(
    df_iris["petal length (cm)"],
    df_iris["petal width (cm)"],
    c=df_iris["cluster_k3"],
    alpha=0.8
)
plt.xlabel("Petal length (cm)")
plt.ylabel("Petal width (cm)")
plt.title("Кластеры k-means (k=3) на признаках лепестков")
plt.show()

## Блок 3. Объяснение работы k-means через центроиды

### 3.1. Координаты центроидов

Получить координаты центров кластеров k-means.
Оформить их в виде таблицы.

In [ ]:
centers_k3 = kmeans_3.cluster_centers_
centers_df_k3 = pd.DataFrame(
    centers_k3,
    columns=iris.feature_names
)
centers_df_k3.index.name = "cluster"
centers_df_k3

### 3.2. Интерпретация центров

Сравнить длины и ширины чашелистиков и лепестков между кластерами и сделать выводы, чем они отличаются.
Эту часть выполнить текстом в отдельных markdown-ячейках (без кода).

## Блок 4. Оценка качества кластеризации методом силуэта

### 4.1. Silhouette score для k = 2…8

Для значений k от 2 до 8:
- обучить k-means;
- вычислить средний коэффициент силуэта;
- сохранить результаты в таблицу и построить график зависимости Silhouette score от k.

In [ ]:
k_values = range(2, 9)
silhouette_scores = []

for k in k_values:
    model = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = model.fit_predict(X)
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)

silhouette_results = pd.DataFrame({
    "k": list(k_values),
    "silhouette_score": silhouette_scores
})
silhouette_results

### 4.2. График зависимости Silhouette score от числа кластеров

Построить график зависимости среднего коэффициента силуэта от числа кластеров k.
По графику выбрать «хорошее» значение k.

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(silhouette_results["k"], silhouette_results["silhouette_score"], marker="o")
plt.xlabel("Число кластеров k")
plt.ylabel("Средний Silhouette score")
plt.title("Silhouette score для разных k (k-means)")
plt.grid(True)
plt.show()

## Блок 5. Подробный силуэт-анализ для выбранного k

### 5.1. Silhouette samples для k = 3

Для выбранного значения k (например, k = 3) вычислить коэффициент силуэта для каждого объекта.
Добавить значения в DataFrame и посмотреть распределение по кластерам.

In [ ]:
silhouette_vals_k3 = silhouette_samples(X, cluster_labels_3)
df_iris["silhouette_k3"] = silhouette_vals_k3
df_iris[["species", "cluster_k3", "silhouette_k3"]].head()

### 5.2. Распределение силуэта по кластерам

Посмотреть средние значения силуэта по кластерам и количество объектов.

In [ ]:
df_iris.groupby("cluster_k3")["silhouette_k3"].agg(["count", "mean"])